# IE402 Assignment 2
### By: 201801015
### 11-10-2020

In [15]:
import numpy as np  
from fractions import Fraction 

def simplex():
    
    # inputs
    n, m = input("Enter number of basis variables and number of constraints seperated by a space:").split(" ")
    n, m = int(n), int(m)
    
    # Coefficients of objective function Z
    c = input("Enter the coefficients of objective function seperated by a space:").split(" ")
    c = np.array(c, dtype = 'int')
    c = np.append(c, np.zeros(m))
    Cj = np.array(np.append(["" ,"", "Cj"],c))
    
    # constraints parameters input
    identity = np.identity(m)
    A = np.array([])
    for i in range(m):
        x = input("Enter constraint parameters seperated by a space:").split(" ")
        A = np.append(A,x)
    A = A.reshape(m,n+2)
    
    # append identity to A
    A = np.c_[A,identity]
    
    # remove leq from matrix
    A = np.delete(A,n,axis = 1)
    
    # remove constraint limit values to get the constraint coefficients array
    b = A[:,n]
    A = np.delete(A,n,axis = 1)
        
    #initial basis variables indexed from 0 consisting of slack variables
    #cb = np.array(c[n+m-1]) 
    B = [i+n for i in range(m)]
    B = np.array(B, dtype ='float')
    B = B.reshape(m,1)  
    
    # cb contains their corresponding coefficients in Z    
    cb = np.zeros(m)   
    cb = cb.reshape(m,1)
    xb = np.transpose([b])
    MR = np.zeros(m)
    MR = MR.reshape(m,1)
    OP = np.zeros(m)
    OP = OP.reshape(m,1)
    
    # combine matrices B and cb 
    table = np.hstack((B, cb))              
    table = np.hstack((table, xb))  
    
    # combine matrices B, cb and xb 
    # finally combine matrix A to form the complete simplex table 
    table = np.hstack((table, A))        
    table = np.hstack((table, MR))  
    table = np.hstack((table, OP)) 
    
    # change the type of table to float 
    table = np.array(table, dtype ='float')  
    # inputs end 

    # if min problem, make this var 1 
    MIN = 0
    
    print('\033[1m'+"\n\nSimplex Algorithm: Tabular Method\n\n") 

    # optimal => reached = 1
    reached = 0  
    # track of number to tables
    itr = 1
    # unbounded LPP
    unbounded = 0
    
    # loop till optimal
    while reached == 0: 

        # calculate Relative profits-> zj - cj 
        i = 0
        rel_prof = [] 
        while i<len(A[0]): 
            rel_prof.append(-c[i] + np.sum(table[:, 1]*table[:, 3 + i])) 
            i = i + 1

        flag = 0
        for profit in rel_prof: 
            if profit<0: 
                flag = 1
                break
            # if all relative profits <= 0 
        if flag == 0: 
            reached = 1

        # kth var will enter the basis 
        newIn = 99999
        for profit in rel_prof: 
            if profit<newIn:
                newIn = profit
        k = rel_prof.index(newIn) 
        minn = 99999
        i = 0; 
        r = -1

        # min ratio test (only positive values) 
        while i<len(table): 
            if (table[:, 2][i]>0 and table[:, 3 + k][i]>0):  
                val = table[:, 2][i]/table[:, 3 + k][i] 
                table[:,3+len(c)][i] = val
                if val<minn: 
                    minn = val 
                    r = i     # leaving variable 
            i+= 1

        print("Iteration: ", end =' ') 
        print(itr) 
        
        for ele in Cj:
            print(ele, end="\t")
        print()
        print("XB \tCB \tb \ta1 \ta2 \ta3 \ta4 \ta5 \tMR \tO/P") 
        # this step is done to change the 0 indexing to 1 indexing for correct display
        table[:, 0] = table[:, 0] + 1
        for row in table: 
            if reached == 1:
                for el in range(len(row)-2):
                    if el == 0:
                        print('x'+str(Fraction(str(row[el])).limit_denominator(100)), end ='\t')
                    else:
                        print(Fraction(str(row[el])).limit_denominator(100), end ='\t')  
                print() 
            else:
                for el in range(len(row)):
                    if el == 0:
                        print('x'+str(Fraction(str(row[el])).limit_denominator(100)), end ='\t')
                    else:
                        print(Fraction(str(row[el])).limit_denominator(100), end ='\t')  
                print() 
        # this step is done to undo the 1 indexing to 0 indexing for correct calculations
        table[:, 0] = table[:, 0] - 1

        print("     Zj - Cj ", end ='\t\t') 
        for profit in rel_prof: 
            print(Fraction(str(profit)).limit_denominator(100), end ='\t') 
        print() 
        
        if reached == 0:
            pivot = table[r][3 + k] 
            print("pivot element: ", end =" ") 
            print(Fraction(pivot+1).limit_denominator(100)) 
        print() 
        
        i = 0
        b_var = table[:, 0] 
        
        # if no min ratio test was performed 
        if r ==-1: 
            unbounded = 1 
            break

        if flag == 0: 
            print("The optimum solution is:") 
            break

        # perform row operations 
        # divide the pivot row with the pivot element 
        table[r, 2:len(table[0])] = table[ 
                r, 2:len(table[0])] / pivot 

        # do row operation on other rows 
        i = 0
        while i<len(table): 
            if i != r: 
                table[i, 2:len(table[0])] = table[i, 2:len(table[0])] - table[i][3 + k] * table[r, 2:len(table[0])] 
            i += 1

        # assign the new basic variable 
        table[r][0] = k 
        table[r][1] = c[k] 

        #print() 
        itr+= 1

    if unbounded == 1: 
        print("UNBOUNDED LPP") 
        exit() 
        
    basis = [] 
    i = 0
    sum = 0
    while i<m: 
        sum += c[int(table[i][0])]*table[i][2] 
        i+= 1
    
    #print 0 to n-1 basis variables
    ###-----!!! ROUND THEM OFF !!!-----###
    for ele in range(n): 
        if ele in table[:,0]:
            print("x"+str(ele+1)+" = "+str(round(table[:,2][np.where(table[:,0] == ele)][0])), end=", ")
        else:
            print("x"+str(ele+1)+" = 0", end=", ")
    print("and Z =", end=" ")
    # if MIN problem make z negative 
    if MIN == 1: 
        print(-Fraction(str(sum)).limit_denominator(100)) 
    else: 
        print(Fraction(str(sum)).limit_denominator(100)) 
    
    print()
    print("Simplex Finished Successfully!") 
    
# call simplex() function
simplex()

Enter number of basis variables and number of constraints seperated by a space:3 2
Enter the coefficients of objective function seperated by a space:1 1 3
Enter constraint parameters seperated by a space:3 2 1 leq 3
Enter constraint parameters seperated by a space:2 1 2 leq 2


Simplex Algorithm: Tabular Method


Iteration:  1
		Cj	1.0	1.0	3.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x4	0	3	3	2	1	1	0	3	0	
x5	0	2	2	1	2	0	1	1	0	
     Zj - Cj 		-1	-1	-3	0	0	
pivot element:  3

Iteration:  2
		Cj	1.0	1.0	3.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x4	0	2	2	3/2	0	1	-1/2	
x3	3	1	1	1/2	1	0	1/2	
     Zj - Cj 		2	1/2	0	0	3/2	

The optimum solution is:
x1 = 0, x2 = 0, x3 = 1.0, and Z = 3

Simplex Finished Successfully!


In [16]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 4
Enter the coefficients of objective function seperated by a space:-4 -3
Enter constraint parameters seperated by a space:2 3 leq 6
Enter constraint parameters seperated by a space:-3 2 leq 3
Enter constraint parameters seperated by a space:0 2 leq 5
Enter constraint parameters seperated by a space:2 1 leq 4


Simplex Algorithm: Tabular Method


Iteration:  1
		Cj	-4.0	-3.0	0.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	6	2	3	1	0	0	0	
x4	0	3	-3	2	0	1	0	0	
x5	0	5	0	2	0	0	1	0	
x6	0	4	2	1	0	0	0	1	
     Zj - Cj 		4	3	0	0	0	0	

The optimum solution is:
x1 = 0, x2 = 0, and Z = 0

Simplex Finished Successfully!


In [18]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 3
Enter the coefficients of objective function seperated by a space:3 2
Enter constraint parameters seperated by a space:2 1 leq 18
Enter constraint parameters seperated by a space:2 3 leq 42
Enter constraint parameters seperated by a space:3 1 leq 24


Simplex Algorithm: Tabular Method


Iteration:  1
		Cj	3.0	2.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	18	2	1	1	0	0	9	0	
x4	0	42	2	3	0	1	0	21	0	
x5	0	24	3	1	0	0	1	8	0	
     Zj - Cj 		-3	-2	0	0	0	
pivot element:  4

Iteration:  2
		Cj	3.0	2.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	2	0	1/3	1	0	-2/3	6	0	
x4	0	26	0	7/3	0	1	-2/3	78/7	0	
x1	3	8	1	1/3	0	0	1/3	24	0	
     Zj - Cj 		0	-1	0	0	1	
pivot element:  4/3

Iteration:  3
		Cj	3.0	2.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x2	2	6	0	1	3	0	-2	18	0	
x4	0	12	0	0	-7	1	4	3	0	
x1	3	6	1	0	-1	0	1	6	0	
     Zj - Cj 		0	0	3	0	-1	
pivot element:  5

Iteration:  4
		Cj	3.0	2.

In [19]:
simplex()

Enter number of basis variables and number of constraints seperated by a space:2 3
Enter the coefficients of objective function seperated by a space:40 30
Enter constraint parameters seperated by a space:1 2 leq 16
Enter constraint parameters seperated by a space:1 1 leq 9
Enter constraint parameters seperated by a space:3 2 leq 24


Simplex Algorithm: Tabular Method


Iteration:  1
		Cj	40.0	30.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	16	1	2	1	0	0	16	0	
x4	0	9	1	1	0	1	0	9	0	
x5	0	24	3	2	0	0	1	8	0	
     Zj - Cj 		-40	-30	0	0	0	
pivot element:  4

Iteration:  2
		Cj	40.0	30.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	8	0	4/3	1	0	-1/3	6	0	
x4	0	1	0	1/3	0	1	-1/3	3	0	
x1	40	8	1	2/3	0	0	1/3	12	0	
     Zj - Cj 		0	-10/3	0	0	40/3	
pivot element:  4/3

Iteration:  3
		Cj	40.0	30.0	0.0	0.0	0.0	
XB 	CB 	b 	a1 	a2 	a3 	a4 	a5 	MR 	O/P
x3	0	4	0	0	1	-4	1	
x2	30	3	0	1	0	3	-1	
x1	40	6	1	0	0	-2	1	
     Zj - Cj 		0	0	0	10	10	

The optimum solution is:
x1 = 6.0, x2 = 2.9